In [104]:
import boto3
import json
import os
import logging
import yaml

In [114]:
prompt_data = "what is the capital of india.?"

In [115]:
# Ensure the "logs" directory exists
log_dir = 'logs'
os.makedirs(log_dir, exist_ok=True)

# logging configuration
logger = logging.getLogger('model_evaluation')
logger.setLevel('DEBUG')

console_handler = logging.StreamHandler()
console_handler.setLevel('DEBUG')

log_file_path = os.path.join(log_dir, 'model_evaluation.log')
file_handler = logging.FileHandler(log_file_path)
file_handler.setLevel('DEBUG')

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

logger.addHandler(console_handler)
logger.addHandler(file_handler)

In [116]:
def load_params(params_path: str) -> dict:
    """Load parameters from a YAML file."""
    try:
        with open(params_path, 'r') as file:
            params = yaml.safe_load(file)
        logger.debug('Parameters retrieved from %s', params_path)
        return params
    except FileNotFoundError:
        logger.error('File not found: %s', params_path)
        raise
    except yaml.YAMLError as e:
        logger.error('YAML error: %s', e)
        raise
    except Exception as e:
        logger.error('Unexpected error: %s', e)
        raise

    

In [117]:
%pwd

'/Users/vivek/work/GIT/assignment/Lakshman Diwaakar'

In [119]:
params = load_params("config.yaml")

2025-01-24 20:32:30,835 - model_evaluation - DEBUG - Parameters retrieved from config.yaml
2025-01-24 20:32:30,835 - model_evaluation - DEBUG - Parameters retrieved from config.yaml
2025-01-24 20:32:30,835 - model_evaluation - DEBUG - Parameters retrieved from config.yaml
2025-01-24 20:32:30,835 - model_evaluation - DEBUG - Parameters retrieved from config.yaml
2025-01-24 20:32:30,835 - model_evaluation - DEBUG - Parameters retrieved from config.yaml


In [120]:
bedrock = boto3.client(
    service_name = "bedrock-runtime",
    aws_access_key_id = params["aws"]["aws_access_key_id"],
    aws_secret_access_key = params["aws"]["secret_access_key"]
)

In [121]:

payload = {
    "inputText": "What is the capital of india", 
    "textGenerationConfig": {
        "maxTokenCount": 30,  # Adjust the maximum token count as needed
        "stopSequences": [],   # List of stop sequences
        "temperature": 0.7,    # Temperature for randomness
        "topP": 1              # Top-p sampling parameter
    }
}


In [122]:
body = json.dumps(payload)

In [123]:
model_id = "amazon.titan-text-lite-v1"

In [ ]:
{
 "modelId": "amazon.titan-text-lite-v1",
 "contentType": "application/json",
 "accept": "application/json",
 "body": "{\"inputText\":\"this is where you place your input text\",\"textGenerationConfig\":{\"maxTokenCount\":4096,\"stopSequences\":[],\"temperature\":0,\"topP\":1}}"
}

In [124]:
respnse  = bedrock.invoke_model(
    modelId = model_id,
    contentType = "application/json",
    accept = "application/json",
    body = body
)

In [125]:
item = respnse["body"].read()

b'{"inputTextTokenCount":6,"results":[{"tokenCount":10,"outputText":"\\nThe capital of India is New Delhi.","completionReason":"FINISH"}]}'

In [126]:
item

b'{"inputTextTokenCount":6,"results":[{"tokenCount":30,"outputText":"\\nNew Delhi is the capital of India. India\'s administrative, legislative, and judicial centers are all located there. The city was established in 1","completionReason":"LENGTH"}]}'

In [127]:
i = json.loads(item)

In [128]:
i["results"][0]["outputText"]

"\nNew Delhi is the capital of India. India's administrative, legislative, and judicial centers are all located there. The city was established in 1"

# working with aws embedding

### preprocessing the data

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%pwd

'/Users/vivek/work/GIT/assignment/Lakshman Diwaakar/aws '

In [3]:
os.chdir("../")

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("data/mutula_fund.csv")

In [6]:
embedding_data = []

In [7]:
clms = list(df.columns)

In [8]:
clms

['Fund Name',
 'Month',
 'Instrument Name',
 'Percentage Allocation',
 'Percentage Allocation Change',
 'Instrument Count Across Funds',
 'Category',
 'Return (%)']

In [9]:
no_of_rows = len(df.iloc[:,0:1])

In [10]:
list(df.iloc[1,:])

['Zerodha Midcap Fund',
 'February 2024',
 'HDFC Large Cap',
 5.25,
 -1.686143078135275,
 1,
 'Large Cap',
 12.69]

In [11]:
df.head()

,Fund Name,Month,Instrument Name,Percentage Allocation,Percentage Allocation Change,Instrument Count Across Funds,Category,Return (%)
0,Zerodha Midcap Fund,January 2024,HDFC Large Cap,6.93,0.000000,3,Small Cap,11.44
1,Zerodha Midcap Fund,February 2024,HDFC Large Cap,5.25,-1.686143,1,Large Cap,12.69
2,Zerodha Midcap Fund,March 2024,HDFC Large Cap,4.92,-0.331520,3,Small Cap,11.53
3,Zerodha Midcap Fund,April 2024,HDFC Large Cap,5.38,0.463300,3,Small Cap,11.51
4,Zerodha Midcap Fund,January 2024,Zerodha Large Cap,18.09,0.000000,1,Mid Cap,11.44


In [12]:
for i in range(no_of_rows):
    data = list(df.iloc[i,:])
    s = ""
    for j in range(len(clms)):
        s+=" "+str(clms[j])+" "+str(data[j])
    embedding_data.append(s)
    if i>4:
        break

In [13]:
for i in embedding_data:
    print(i)

 Fund Name Zerodha Midcap Fund Month January 2024 Instrument Name HDFC Large Cap Percentage Allocation 6.93 Percentage Allocation Change 0.0 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.44
 Fund Name Zerodha Midcap Fund Month February 2024 Instrument Name HDFC Large Cap Percentage Allocation 5.25 Percentage Allocation Change -1.686143078135275 Instrument Count Across Funds 1 Category Large Cap Return (%) 12.69
 Fund Name Zerodha Midcap Fund Month March 2024 Instrument Name HDFC Large Cap Percentage Allocation 4.92 Percentage Allocation Change -0.3315197881022982 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.53
 Fund Name Zerodha Midcap Fund Month April 2024 Instrument Name HDFC Large Cap Percentage Allocation 5.38 Percentage Allocation Change 0.4633003382439566 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.51
 Fund Name Zerodha Midcap Fund Month January 2024 Instrument Name Zerodha Large Cap Percentage Allocation 18.09 Percentag

In [41]:
data = [{"id": f"item-{i+1}", "text": embedding_data[i]} for i in range(len(embedding_data))]

In [42]:
data

[{'id': 'item-1',
  'text': ' Fund Name Zerodha Midcap Fund Month January 2024 Instrument Name HDFC Large Cap Percentage Allocation 6.93 Percentage Allocation Change 0.0 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.44'},
 {'id': 'item-2',
  'text': ' Fund Name Zerodha Midcap Fund Month February 2024 Instrument Name HDFC Large Cap Percentage Allocation 5.25 Percentage Allocation Change -1.686143078135275 Instrument Count Across Funds 1 Category Large Cap Return (%) 12.69'},
 {'id': 'item-3',
  'text': ' Fund Name Zerodha Midcap Fund Month March 2024 Instrument Name HDFC Large Cap Percentage Allocation 4.92 Percentage Allocation Change -0.3315197881022982 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.53'},
 {'id': 'item-4',
  'text': ' Fund Name Zerodha Midcap Fund Month April 2024 Instrument Name HDFC Large Cap Percentage Allocation 5.38 Percentage Allocation Change 0.4633003382439566 Instrument Count Across Funds 3 Category Small Cap Return (%) 11

### embedding generation

In [14]:
import boto3
import json

In [15]:
bedrock = boto3.client(
    service_name = "bedrock-runtime",
    region_name = "us-east-1"
)


In [17]:
print(embedding_data[0:3])

[' Fund Name Zerodha Midcap Fund Month January 2024 Instrument Name HDFC Large Cap Percentage Allocation 6.93 Percentage Allocation Change 0.0 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.44', ' Fund Name Zerodha Midcap Fund Month February 2024 Instrument Name HDFC Large Cap Percentage Allocation 5.25 Percentage Allocation Change -1.686143078135275 Instrument Count Across Funds 1 Category Large Cap Return (%) 12.69', ' Fund Name Zerodha Midcap Fund Month March 2024 Instrument Name HDFC Large Cap Percentage Allocation 4.92 Percentage Allocation Change -0.3315197881022982 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.53']


In [34]:
body = json.dumps({
    "inputText":embedding_data[0]
})

TITAN_EMBEDDING_MODEL = "amazon.titan-embed-text-v1"
content_type = "application/json"
accept = "*/*"

In [ ]:
response = bedrock.invoke_model(
    body = body,
    modelId = TITAN_EMBEDDING_MODEL,
    contentType = content_type,
    accept = accept
)

In [23]:
item = response["body"].read()

In [24]:
i = json.loads(item)

In [25]:
len(i["embedding"])

1536

In [26]:
i["embedding"]

[0.26171875,
 0.83984375,
 -0.1552734375,
 0.333984375,
 -0.12353515625,
 -0.78515625,
 0.7265625,
 -0.0014801025390625,
 -1.34375,
 -0.33203125,
 0.423828125,
 0.75390625,
 0.07275390625,
 0.5546875,
 -0.349609375,
 -0.10595703125,
 0.625,
 -0.07177734375,
 -0.01129150390625,
 -0.43359375,
 -0.55078125,
 -0.1748046875,
 0.416015625,
 0.49609375,
 -0.6953125,
 -0.298828125,
 0.4375,
 0.443359375,
 -0.3984375,
 0.07958984375,
 0.037109375,
 0.8828125,
 0.5859375,
 -0.2890625,
 -0.234375,
 -0.70703125,
 0.1689453125,
 -1.0,
 0.0264892578125,
 0.498046875,
 0.11865234375,
 -0.2412109375,
 0.9140625,
 0.5234375,
 -0.6953125,
 -0.2060546875,
 -0.031982421875,
 -0.009033203125,
 -0.296875,
 -0.2265625,
 -0.01123046875,
 -0.80859375,
 -0.00531005859375,
 0.37109375,
 -0.50390625,
 1.1796875,
 0.455078125,
 0.7421875,
 -0.09228515625,
 0.035400390625,
 0.060546875,
 0.30859375,
 -0.267578125,
 0.4296875,
 0.314453125,
 0.65234375,
 0.69140625,
 -0.05078125,
 -0.34765625,
 0.6484375,
 0.4023437

In [37]:
def generate_titan_embedding(text):
    """
    Retures the Embedding data for a given text
    Args:
        text -  the input text
    Returs:
        Embedding vector (dimension 1536)
    """
    response = bedrock.invoke_model(
        modelId=TITAN_EMBEDDING_MODEL,
        contentType="application/json",
        accept="application/json",
        body=json.dumps({"inputText": text})
    )
    result = json.loads(response["body"].read())
    return result["embedding"]  # Embedding vector

In [39]:
d2 = generate_titan_embedding(embedding_data[0])

In [40]:
d2

[0.26171875,
 0.83984375,
 -0.1552734375,
 0.333984375,
 -0.12353515625,
 -0.78515625,
 0.7265625,
 -0.0014801025390625,
 -1.34375,
 -0.33203125,
 0.423828125,
 0.75390625,
 0.07275390625,
 0.5546875,
 -0.349609375,
 -0.10595703125,
 0.625,
 -0.07177734375,
 -0.01129150390625,
 -0.43359375,
 -0.55078125,
 -0.1748046875,
 0.416015625,
 0.49609375,
 -0.6953125,
 -0.298828125,
 0.4375,
 0.443359375,
 -0.3984375,
 0.07958984375,
 0.037109375,
 0.8828125,
 0.5859375,
 -0.2890625,
 -0.234375,
 -0.70703125,
 0.1689453125,
 -1.0,
 0.0264892578125,
 0.498046875,
 0.11865234375,
 -0.2412109375,
 0.9140625,
 0.5234375,
 -0.6953125,
 -0.2060546875,
 -0.031982421875,
 -0.009033203125,
 -0.296875,
 -0.2265625,
 -0.01123046875,
 -0.80859375,
 -0.00531005859375,
 0.37109375,
 -0.50390625,
 1.1796875,
 0.455078125,
 0.7421875,
 -0.09228515625,
 0.035400390625,
 0.060546875,
 0.30859375,
 -0.267578125,
 0.4296875,
 0.314453125,
 0.65234375,
 0.69140625,
 -0.05078125,
 -0.34765625,
 0.6484375,
 0.4023437

### integrating pinecone


In [27]:
from dotenv import load_dotenv
load_dotenv()
import os
api_ = os.getenv("PINECONE")

In [28]:
from langchain_community.retrievers import pinecone_hybrid_search
from pinecone import Pinecone, ServerlessSpec
index_name = "testa"

#initilaise the pipnecone client
pc = Pinecone(api_key=api_)




/opt/miniconda3/envs/intenr_1/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [29]:
#creatinng the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension=1536,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [51]:
index = pc.Index(index_name)
index

In [44]:
data

[{'id': 'item-1',
  'text': ' Fund Name Zerodha Midcap Fund Month January 2024 Instrument Name HDFC Large Cap Percentage Allocation 6.93 Percentage Allocation Change 0.0 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.44'},
 {'id': 'item-2',
  'text': ' Fund Name Zerodha Midcap Fund Month February 2024 Instrument Name HDFC Large Cap Percentage Allocation 5.25 Percentage Allocation Change -1.686143078135275 Instrument Count Across Funds 1 Category Large Cap Return (%) 12.69'},
 {'id': 'item-3',
  'text': ' Fund Name Zerodha Midcap Fund Month March 2024 Instrument Name HDFC Large Cap Percentage Allocation 4.92 Percentage Allocation Change -0.3315197881022982 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.53'},
 {'id': 'item-4',
  'text': ' Fund Name Zerodha Midcap Fund Month April 2024 Instrument Name HDFC Large Cap Percentage Allocation 5.38 Percentage Allocation Change 0.4633003382439566 Instrument Count Across Funds 3 Category Small Cap Return (%) 11

In [43]:
def upsert_to_pinecone(data):
    """
    Data should be a list of dictionaries, where each dictionary contains:
    - 'id': unique ID for the item
    - 'text': the text to embed
    """
    for item in data:
        text = item["text"]
        embedding = generate_titan_embedding(text)
        index.upsert(vectors=[(item["id"], embedding)])

In [45]:
upsert_to_pinecone(data)

In [52]:
def query_pinecone(query_text, top_k=5):
    """
    Query Pinecone index with a text query and return results sorted by score in descending order.
    
    Args:
        query_text (str): The text to query.
        top_k (int): The number of top results to retrieve.

    Returns:
        list: List of results sorted by score in descending order.
    """
    # Generate embedding for the query
    query_embedding = generate_titan_embedding(query_text)
    
    # Query Pinecone index
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    
    # Sort results by score in descending order (if not already sorted)
    sorted_results = sorted(results["matches"], key=lambda x: x["score"], reverse=True)
    
    return sorted_results

In [53]:
q = "Zerodha Midcap Fund Month January 2024 Zerodha Large Cap Mid Cap Return (%) 11.44"
output = query_pinecone(q, top_k=2)

In [48]:
output

{'matches': [{'id': 'item-1', 'score': 236.473724, 'values': []},
             {'id': 'item-5', 'score': 235.988983, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [54]:
output

[{'id': 'item-1', 'score': 236.70433, 'values': []},
 {'id': 'item-5', 'score': 236.11409, 'values': []}]

In [56]:
ids_to_retrieve = [result["id"] for result in output]

In [57]:
ids_to_retrieve

['item-1', 'item-5']

In [65]:
data_dict = {item['id']: item['text'] for item in data}

In [66]:
data_dict

{'item-1': ' Fund Name Zerodha Midcap Fund Month January 2024 Instrument Name HDFC Large Cap Percentage Allocation 6.93 Percentage Allocation Change 0.0 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.44',
 'item-2': ' Fund Name Zerodha Midcap Fund Month February 2024 Instrument Name HDFC Large Cap Percentage Allocation 5.25 Percentage Allocation Change -1.686143078135275 Instrument Count Across Funds 1 Category Large Cap Return (%) 12.69',
 'item-3': ' Fund Name Zerodha Midcap Fund Month March 2024 Instrument Name HDFC Large Cap Percentage Allocation 4.92 Percentage Allocation Change -0.3315197881022982 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.53',
 'item-4': ' Fund Name Zerodha Midcap Fund Month April 2024 Instrument Name HDFC Large Cap Percentage Allocation 5.38 Percentage Allocation Change 0.4633003382439566 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.51',
 'item-5': ' Fund Name Zerodha Midcap Fund Month January 2024 Ins

In [67]:

def retrieve_data_by_id(ids):
    """
    Retrieve data from the data dictionary using the list of IDs.
    
    Args:
        ids (list): List of IDs to retrieve.
    
    Returns:
        list: List of corresponding data items.
    """
    results = [data_dict.get(id, "ID not found") for id in ids]
    return results

In [68]:
retrieved_data = retrieve_data_by_id(ids_to_retrieve)

In [69]:
retrieved_data

[' Fund Name Zerodha Midcap Fund Month January 2024 Instrument Name HDFC Large Cap Percentage Allocation 6.93 Percentage Allocation Change 0.0 Instrument Count Across Funds 3 Category Small Cap Return (%) 11.44',
 ' Fund Name Zerodha Midcap Fund Month January 2024 Instrument Name Zerodha Large Cap Percentage Allocation 18.09 Percentage Allocation Change 0.0 Instrument Count Across Funds 1 Category Mid Cap Return (%) 11.44']